<center style="font-size: 2em; font-weight: bold;"> Base de datos Mincer Peruano </center>
<center style="font-size: 2em; font-weight: bold;"> Manejo de base de datos  </center>
<left style="font-size: 4em; font-weight: bold;"> Autor: Edinson Tolentino </left>

# Inicio

In [1]:
import pandas
import numpy
import os
import sys
#import pyreadstat

In [2]:
import codigos_enaho

## Cargar Data

In [3]:
data_enaho       = "C:/Users/et396/Dropbox/BASES/ENAHO"
output           = "C:/Users/et396/Dropbox/Docencia/Educate/Econometria/S1/Data"


In [4]:
# Seleccion de variables de la base de empleo
variables_500 = ['conglome', 'vivienda', 'hogar', 'codperso','p301a','ocu500','p208a',
                 'i524a1', 'd529t', 'i530a', 'd536', 'i538a1', 'd540t', 'i541a', 'd543', 'd544t',
                 'p207','p203','ubigeo', 'p209']

In [ ]:
fechas =[2019]
for x in fechas:
    
    # informacion de carpetas 
    year             = f"{x}"
    modulo_empleo    = f"enaho01a-{x}-500.dta" # survey labor force
    modulo_educacion = f"enaho01a-{x}-300.dta" # survey education
    modulo_sumaria   = f"sumaria-{x}.dta"      # survey poverty


    data_500      = os.path.join(data_enaho, year, modulo_empleo)
    data_300      = os.path.join(data_enaho, year, modulo_educacion)
    data_340      = os.path.join(data_enaho, year, modulo_sumaria)


    # Carga de data
    data_empleo      = pandas.read_stata(data_500 , columns = variables_500 ) # filtro de variables

In [ ]:
data_empleo['p209'].value_counts()
#data_empleo['p209'].info()


In [5]:

fechas =[2019, 2018, 2020, 2021]
for x in fechas:
    
    # informacion de carpetas 
    year             = f"{x}"
    modulo_empleo    = f"enaho01a-{x}-500.dta" # survey labor force
    modulo_educacion = f"enaho01a-{x}-300.dta" # survey education
    modulo_sumaria   = f"sumaria-{x}.dta"      # survey poverty


    data_500      = os.path.join(data_enaho, year, modulo_empleo)
    data_300      = os.path.join(data_enaho, year, modulo_educacion)
    data_340      = os.path.join(data_enaho, year, modulo_sumaria)


    # Carga de data
    data_empleo      = pandas.read_stata(data_500 , columns = variables_500 ) # filtro de variables
    
    data_educacion   = pandas.read_stata(data_300, convert_categoricals=False )
    data_sumaria     = pandas.read_stata(data_340)
    
    # Informacion de extraccion de variables
    frame_empleo      = codigos_enaho.function_r500(data_empleo)
    frame_empleo      = frame_empleo[(frame_empleo.p203=='jefe/jefa')]    # solo jefe de hogar
    #frame_empleo      = frame_empleo[(frame_empleo.r3<=2)]                # Solo PEA
    frame_empleo      = frame_empleo[(frame_empleo.r3==1)]                # Solo PEA Ocupada
    #frame_empleo      = frame_empleo[(frame_empleo.r6>0)]                # Salarios positivos
    
    frame_educacion   = codigos_enaho.function_r300(data_educacion)
    frame_sumaria     = codigos_enaho.function_rsumaria(data_sumaria)
    
    # Generacion y filtro de variables
    var_empleo = [
        'rcod_persona',
        'rcod_hogar',
        'rmujer', 
        'r6', 
        'lnr6',
        'rmu', # desempleo
        'rneduca',
        'r3',
        'redad',
        'redadsq',
        'rDpto',
        'rpareja',
    ]

    var_educacion = [
        'rcod_persona',
        'rcod_hogar',
        'reduca', 
    ]

    var_sumaria = [ 
        'rcod_hogar',
        'rpobre', 
        'rmiembro',
        'rgasto',
    ]
    
    # Union de variables
    base_empleo    = frame_empleo[var_empleo]
    # print(base_empleo.shape)
    base_educacion = frame_educacion[var_educacion]
    # print(base_educacion.shape) 
    base_pobreza   = frame_sumaria[var_sumaria]
    # print(base_pobreza.shape)
    
    base_final =pandas.merge(base_empleo, base_educacion, how='inner')
    base_final =pandas.merge(base_final , base_pobreza  , how='inner')
    print()
    print("--------------------- informacion de cantidad de datos y variables year:" f"{x}" )
    # print(base_final.shape)
    base_final = codigos_enaho.funcion_rfiltros(base_final)
    print(base_final.shape) 
    #base_final.columns
    print(base_final.sample(3).head().T)
    
    os.chdir(output) # exportar los resultados hacia una base de datos
    #base_final.to_excel(f"Mincer_{x}.xlsx")
    base_final.to_csv(f"Mincer_{x}.csv") # Informacion de csv
    base_final.to_stata(f"Mincer_{x}.dta") # Informacion de stata
    #base_final.to_spss(f"Mincer_{x}.sav") # Informacion de spss
    

C:\Users\et396\.conda\envs\python_last\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:48: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['rneduca'] = frame['p301a'].replace({"sin nivel": "inicial",
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:70: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['pareja'] = frame['p209'].replace({"casado(a)": "pare


--------------------- informacion de cantidad de datos y variables year:2019
(25074, 16)
                      6270           18286          6617 
rcod_persona  0081251431101  0075970081101  0089410351101
rcod_hogar      00812514311    00759700811    00894103511
rmujer                    0              0              1
r6                  5040.75    1995.916667          397.0
lnr6                8.52531       7.598859       5.983936
rmu                       0              0              0
rneduca            primaria          Univ.       primaria
r3                        1              1              1
redad                    44             34             36
redadsq                1936           1156           1296
rDpto                  Lima           Lima          Piura
rpareja                   1              0              1
reduca                  0.0           16.0            0.0
rpobre                    1              0              1
rmiembro                  6             

C:\Users\et396\.conda\envs\python_last\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:48: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['rneduca'] = frame['p301a'].replace({"sin nivel": "inicial",
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:70: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['pareja'] = frame['p209'].replace({"casado(a)": "pare


--------------------- informacion de cantidad de datos y variables year:2018
(27486, 16)
                      31930          18501          15343
rcod_persona  0111820671101  0084601591101  0078260181101
rcod_hogar      01118206711    00846015911    00782601811
rmujer                    0              1              1
r6                   1485.0     896.916667    5668.333333
lnr6                7.30317       6.798963        8.64265
rmu                       0              0              0
rneduca             tecnica          Univ.          Univ.
r3                        1              1              1
redad                    29             33             27
redadsq                 841           1089            729
rDpto         Madre de Dios  Madre de Dios           Lima
rpareja                   1              0              0
reduca                 14.0           14.0           17.0
rpobre                    0              0              0
rmiembro                  4             

C:\Users\et396\.conda\envs\python_last\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:48: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['rneduca'] = frame['p301a'].replace({"sin nivel": "inicial",
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:70: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['pareja'] = frame['p209'].replace({"casado(a)": "pare


--------------------- informacion de cantidad de datos y variables year:2020
(23545, 16)
                      10920          17203          26588
rcod_persona  0058540661101  0195071381101  0194680041101
rcod_hogar      00585406611    01950713811    01946800411
rmujer                    0              0              1
r6              2624.083333     388.833333    2761.166667
lnr6               7.872487       5.963151       7.923409
rmu                       0              0              0
rneduca             tecnica     secundaria     secundaria
r3                        1              1              1
redad                    58             47             46
redadsq                3364           2209           2116
rDpto             Cajamarca           Puno           Puno
rpareja                   1              1              0
reduca                 16.0           11.0            9.0
rpobre                    0              1              0
rmiembro                  2             

C:\Users\et396\.conda\envs\python_last\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:48: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['rneduca'] = frame['p301a'].replace({"sin nivel": "inicial",
C:\Users\et396\Dropbox\Docencia\Educate\Econometria\S1\Data\codigos_enaho.py:70: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  frame['pareja'] = frame['p209'].replace({"casado(a)": "pare


--------------------- informacion de cantidad de datos y variables year:2021
(24774, 16)
                      11358          2755           4644 
rcod_persona  0160310851101  0072360111101  0089840101101
rcod_hogar      01603108511    00723601111    00898401011
rmujer                    1              0              0
r6               991.166667          406.0          994.5
lnr6               6.898883       6.006353        6.90224
rmu                       0              0              0
rneduca             tecnica       primaria        tecnica
r3                        1              1              1
redad                    21             60             34
redadsq                 441           3600           1156
rDpto             Cajamarca    La Libertad           Puno
rpareja                   0              1              1
reduca                 13.0            3.0           14.0
rpobre                    0              0              1
rmiembro                  2             

In [ ]:
# Analisis de datos

In [ ]:
base_final.info()

In [6]:
base_final[['r6','rmu','rmujer','rpareja']].describe()

,r6,rmu,rmujer,rpareja
count,24774.000000,24774.0,24774.000000,24774.000000
mean,1443.894301,0.0,0.283119,0.657221
std,1745.175787,0.0,0.450523,0.474648
min,0.000000,0.0,0.000000,0.000000
25%,495.000000,0.0,0.000000,0.000000
50%,996.041667,0.0,0.000000,1.000000
75%,1761.937500,0.0,1.000000,1.000000
max,52063.250000,0.0,1.000000,1.000000
